In [1]:
import polars as pl
import polars_ds as pds
import numpy as np

# This notebook illustrates the basic usage of this package

You need to create an environment with this package installed to run this notebook. (usually latest version)

# Num Extensions

In [2]:
size = 10_000
df = pl.DataFrame({
    "f": np.sin(list(range(size)))
    , "time_idx": range(size)
    , "dummy": ["a"] * (size // 2) + ["b"] * (size // 2)
    , "actual": np.round(np.random.random(size=size)).astype(np.int32)
    , "predicted": np.random.random(size=size)
    , "dummy_groups":["a"] * (size//2) + ["b"] * (size//2) 
}).with_columns(
    pds.random(0., 1.).alias("x1")
    , pds.random(0., 1.).alias("x2")
    , pds.random(0., 1.).alias("x3")
    , pds.random(0., 1.).alias("a")
    , pds.random(0., 1.).alias("b")
).with_columns(
    y = pl.col("x1") * 0.15 + pl.col("x2") * 0.3 - pl.col("x3") * 1.5 + pds.random() * 0.0001,
    y2 = pl.col("x1") * 0.13 + pl.col("x2") * 0.45 - pl.col("x3") * 0.1 + pds.random() * 0.0001
)
df.head()

f,time_idx,dummy,actual,predicted,dummy_groups,x1,x2,x3,a,b,y,y2
f64,i64,str,i32,f64,str,f64,f64,f64,f64,f64,f64,f64
0.0,0,"""a""",0,0.870124,"""a""",0.499676,0.116938,0.587946,0.131978,0.324241,-0.771857,0.058872
0.841471,1,"""a""",0,0.626794,"""a""",0.791879,0.094017,0.374373,0.489786,0.504696,-0.414556,0.107824
0.909297,2,"""a""",0,0.362253,"""a""",0.526741,0.544468,0.649537,0.824565,0.414665,-0.73191,0.248602
0.14112,3,"""a""",1,0.236681,"""a""",0.196969,0.264493,0.736293,0.42734,0.734808,-0.995451,0.071098
-0.756802,4,"""a""",0,0.147215,"""a""",0.786499,0.964315,0.764042,0.788627,0.2992,-0.738782,0.459861


In [3]:
# Column-wise Jaccard Similarity. Result should be 0 as they are distinct
df.select(
    pds.jaccard_col("x1", pl.col("x2"))
)

x1
f64
0.0


In [4]:
# FFT. First is real part, second is complex part
# By default, this behaves the same as np's rfft, which returns a non-redundant 
# compact representation of fft output.
df.select(
    pds.rfft("f")
).head()

f
"array[f64, 2]"
"[1.939505, 0.0]"
"[1.939506, 0.000209]"
"[1.939508, 0.000418]"
"[1.939512, 0.000627]"
"[1.939518, 0.000835]"


In [5]:
# FFT. But return the full length
df.select(
    pds.rfft("f", return_full=True)
).shape

(10000, 1)

In [6]:
# Multiple Convolutions at once
# Modes: `same`, `left` (left-aligned same), `right` (right-aligned same), `valid` or `full`
# Method: `fft`, `direct`
# Currently slower than SciPy but provides parallelism because of Polars
df.select(
    pds.convolve("f", [-1, 0, 0, 0, 1], mode = "full", method = "fft"), # column f with the kernel given here
    pds.convolve("a", [-1, 0, 0, 0, 1], mode = "full", method = "direct"),
    pds.convolve("b", [-1, 0, 0, 0, 1], mode = "full", method = "direct"),
).head()

f,a,b
f64,f64,f64
1.3944e-15,-0.131978,-0.324241
-0.841471,-0.489786,-0.504696
-0.909297,-0.824565,-0.414665
-0.14112,-0.42734,-0.734808
0.756802,-0.656649,0.025041


In [7]:
# Linear Regression
df.select(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    )
)

coeffs
list[f64]
"[-0.497998, -0.333503]"


In [8]:
# Linear Regression, multi-target
df.select(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = [pl.col("y"), pl.col("y2")],
        add_bias=False
    )
).unnest("coeffs")

target_0,target_1
list[f64],list[f64]
"[-0.497998, -0.333503]","[0.08684, 0.407807]"


In [9]:
df.select(
    pds.lin_reg_report(
        # formulaic input is also available for lstsq related queries, 
        # or you can always use polars expressions, e.g. pl.col('x1') + 1, pl.col('x2').exp(), pl.col('x3').sin()
        "ln(x1+1)", "exp(x2)", "sin(x3)",
        target = "y",
        add_bias = True
    ).alias("report")
).unnest("report")

features,beta,std_err,t,p>|t|,0.025,0.975
str,f64,f64,f64,f64,f64,f64
"""ln(x1+1)""",0.219173,0.001673,131.006017,0.0,0.215893,0.222452
"""exp(x2)""",0.175514,0.000678,258.836417,0.0,0.174185,0.176843
"""sin(x3)""",-1.741122,0.001334,-1304.782909,0.0,-1.743738,-1.738507
"""__bias__""",-0.110494,0.001485,-74.408752,0.0,-0.113404,-0.107583


In [10]:
df.select(
    "dummy",
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    ).over(pl.col("dummy"))
)

dummy,coeffs
str,list[f64]
"""a""","[-0.491841, -0.342778]"
"""a""","[-0.491841, -0.342778]"
"""a""","[-0.491841, -0.342778]"
"""a""","[-0.491841, -0.342778]"
"""a""","[-0.491841, -0.342778]"
…,…
"""b""","[-0.504246, -0.324211]"
"""b""","[-0.504246, -0.324211]"
"""b""","[-0.504246, -0.324211]"


In [11]:
# If you want prediction and residue instead of coefficients
df.select(
    "x1",
    "x2",
    "y",
    pds.lin_reg(
        "x1", pl.col("x2"),
        target = "y",
        add_bias=False, 
        return_pred=True
    ).alias("prediction")
).unnest("prediction").head()

x1,x2,y,pred,resid
f64,f64,f64,f64,f64
0.499676,0.116938,-0.771857,-0.287837,-0.48402
0.791879,0.094017,-0.414556,-0.425709,0.011154
0.526741,0.544468,-0.73191,-0.443897,-0.288013
0.196969,0.264493,-0.995451,-0.186299,-0.809152
0.786499,0.964315,-0.738782,-0.713276,-0.025506


In [12]:
df.group_by("dummy").agg(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    )
)

dummy,coeffs
str,list[f64]
"""a""","[-0.491841, -0.342778]"
"""b""","[-0.504246, -0.324211]"


In [13]:
# Lasso
df.group_by("dummy").agg(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        l1_reg = 0.1,
        add_bias=False
    )
)

dummy,coeffs
str,list[f64]
"""a""","[-0.329471, -0.157246]"
"""b""","[-0.338692, -0.144217]"


In [14]:
# R2 metric of lasso regressions on each group
df.group_by("dummy").agg(
    pds.query_r2(
        actual = pl.col("y"),
        pred = pds.lin_reg(
            pl.col("x1"), pl.col("x2"),
            target = pl.col("y"),
            l1_reg = 0.1,
            return_pred = True,
            add_bias=False
        ).struct.field("pred")
    ).alias("lasso_r2")
)

dummy,lasso_r2
str,f64
"""a""",-0.539511
"""b""",-0.52702


In [15]:
# Rolling regression
df.select(
    "y",
    "x1",
    "x2",
    pds.rolling_lin_reg(
        "x1", "x2",
        target = "y",
        window_size = 5,
        null_policy = "zero"
    ).alias("result")
).unnest("result")

y,x1,x2,coeffs,pred
f64,f64,f64,list[f64],f64
-0.771857,0.499676,0.116938,null,null
-0.414556,0.791879,0.094017,null,null
-0.73191,0.526741,0.544468,null,null
-0.995451,0.196969,0.264493,null,null
-0.738782,0.786499,0.964315,"[-0.714089, -0.473885]",-1.018605
…,…,…,…,…
0.126856,0.126946,0.964879,"[-1.072946, 0.029]",-0.108224
-1.313661,0.899251,0.051631,"[-1.268071, 0.358157]",-1.121822
-0.550832,0.911417,0.220975,"[-1.06123, 0.355866]",-0.888586


In [16]:
# Conditional Entropy, should be 0 because x1 is an ID
df.select(
    pds.query_cond_entropy("y", "x1")
)

y
f64
-0.0


In [17]:
# Only want singular values (principal values?)
df.select(
    pds.singular_values("a", "b", "x1")
)

a
list[f64]
"[29.123673, 28.887074, 28.317706]"


In [18]:
# Singular values + The principal components
df.select(
    pds.pca("a", "b")
).unnest("a")

singular_value,weight_vector
f64,list[f64]
29.0503,"[0.835278, -0.549827]"
28.458703,"[0.549827, 0.835278]"


In [19]:
# PC1
df.select(
    pds.principal_components("a", "b", k =1).alias("principal_components")
).unnest("principal_components").head()

pc1
f64
-0.210686
-0.011035
0.318099
-0.189717
0.351567


# ML Metrics

In [20]:
df.group_by("dummy_groups").agg(
    pds.query_l2("actual", "predicted").alias("l2"),
    pds.query_log_loss("actual", "predicted").alias("log loss"),
    pds.query_binary_metrics(actual="actual", pred="predicted").alias("combo")
).unnest("combo")


dummy_groups,l2,log loss,precision,recall,f,average_precision,roc_auc
str,f64,f64,f64,f64,f64,f64,f64
"""b""",0.334322,1.001752,0.490312,0.506743,0.498392,0.483651,0.496841
"""a""",0.33018,0.994606,0.508013,0.500988,0.504476,0.513971,0.508563


# Str Extension

In [21]:
size = 100_000
df2 = pl.DataFrame({
    "sen":["Hello, world! I'm going to church."] * size,
    "word":["words", "word"] * (size //2)
})
df2.head()

sen,word
str,str
"""Hello, world! I'm going to chu…","""words"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""words"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""words"""


In [22]:
# Tokenize
df2.select(
    pds.str_tokenize(pl.col("sen").str.to_lowercase()).explode().unique()
)

sen
str
"""world"""
"""going"""
"""church"""
"""to"""
"""hello"""


In [23]:
df2.select(
    pds.str_tokenize(pl.col("sen").str.to_lowercase(), stem=True).explode().unique()
)

sen
str
""""""
"""hello"""
"""world"""
"""go"""
"""church"""


In [24]:
df2.select(
    pds.str_leven("word", pl.lit("world"))
).head()

word
u32
2
1
2
1
2


In [25]:
# Damerau-Levenshtein
df2.select(
    pds.str_d_leven("word", pl.lit("world"))
).head()

word
u32
2
1
2
1
2


In [26]:
df2.select( # column "word" vs. the word "world"
    pds.str_leven("word", pl.lit("world"), return_sim = True)
).head()

word
f64
0.6
0.8
0.6
0.8
0.6


In [27]:
df2.filter(
    # This is way faster than computing ditance and then doing a filter
    pds.filter_by_levenshtein(pl.col("word"), pl.lit("world"), 1) # <= 1. 
).head()

sen,word
str,str
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""


In [28]:
df = pl.DataFrame({
    "word":["apple", "banana", "pineapple", "asasasas", "sasasass"],
    "other_data": [1,2,3,4,5]
})
gibberish = ["asasasa", "sasaaasss", "asdasadadfa"]

In [29]:
df.select(
    # Nearest string
    pds.str_nearest("word", word = "banana")
)

word
str
"""banana"""


In [30]:
df.filter(
    # Filters to words that are similar to any word in vocab
    pds.similar_to_vocab(
        pl.col("word"),
        vocab = gibberish,
        threshold = 0.5,
        metric = "lv", # Levenshtein similarity. Other options: dleven, osa, jw
        strategy = "any" # True if the word is similar to any word in vocab. Other options: "all", "avg"
    )
)

word,other_data
str,i64
"""asasasas""",4
"""sasasass""",5


In [31]:
df.select(
    pds.str_leven("word", pl.lit("asasasa"), return_sim=True).alias("asasasa"),
    pds.str_leven("word", pl.lit("sasaaasss"), return_sim=True).alias("sasaaasss"),
    pds.str_leven("word", pl.lit("asdasadadfa"), return_sim=True).alias("asdasadadfa"),
    pds.str_fuzz("word", pl.lit("apples")).alias("LCS based Fuzz match - apples"),
    pds.str_osa("word", pl.lit("apples"), return_sim=True).alias("Optimal String Alignment - apples"),
    pds.str_jw("word", pl.lit("apples")).alias("Jaro-Winkler - apples"),
)


asasasa,sasaaasss,asdasadadfa,LCS based Fuzz match - apples,Optimal String Alignment - apples,Jaro-Winkler - apples
f64,f64,f64,f64,f64,f64
0.142857,0.111111,0.090909,0.833333,0.833333,0.966667
0.428571,0.333333,0.272727,0.166667,0.0,0.444444
0.111111,0.111111,0.090909,0.555556,0.444444,0.5
0.875,0.666667,0.545455,0.25,0.25,0.527778
0.75,0.777778,0.454545,0.25,0.25,0.527778


# Stats Extension

In [32]:
import numpy as np

df = pl.DataFrame({
    "a": [None, None] + list(np.random.normal(size = 998))
})
df.head()

a
f64
null
null
0.620102
0.354738
-1.146126


In [33]:
# Genenrate random numbers, respecting null positions in reference column (pl.col("a"))
df.with_columns(
    pds.random_normal(mean = 0.5, std = 1.0).alias("random_normal"),
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_normal(mean = 0.5, std = 1.0).alias("random_normal")
    ).alias("random_normal_that_respects_null_of_a")
).head()

a,random_normal,random_normal_that_respects_null_of_a
f64,f64,f64
null,1.48806,null
null,1.578787,null
0.620102,1.197807,1.4971
0.354738,1.315951,1.310793
-1.146126,0.020791,-0.163486


In [34]:
# Genenrate random string
df.with_columns(
    pds.random_str(min_size = 1, max_size = 5).alias("random_str"),
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_str(min_size = 1, max_size = 5)
    ).alias("random_str_that_respects_null_of_a")
).head()

a,random_str,random_str_that_respects_null_of_a
f64,str,str
null,"""Tyl""",null
null,"""9o""",null
0.620102,"""j8""","""tIAdD"""
0.354738,"""aqGdl""","""yqXA"""
-1.146126,"""9y""","""BaOP"""


In [35]:
# Genenrate fixed size random string, while respecting column a's nulls
df.with_columns(
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_str(min_size = 5, max_size = 5)
    ).alias("random_str")
).head()

a,random_str
f64,str
null,null
null,null
0.620102,"""Grcv7"""
0.354738,"""FnKt9"""
-1.146126,"""CifSX"""


In [36]:
df.with_columns(
    # Sample from a normal distribution, using reference column "a" 's mean and std
    pds.random_normal(pl.col("a").mean(), pl.col("a").std()).alias("test1") 
    # Sample from uniform distribution, with low = 0 and high = "a"'s max, and respect the nulls in "a"
    , pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random(lower = 0., upper = pl.col("a").max()).alias("test2")
    )
).with_columns(
    # Add a random pertubation to test1
    pds.perturb("test1", epsilon=0.001).alias("test1_perturbed")
).head()

a,test1,literal,test1_perturbed
f64,f64,f64,f64
null,-1.354802,null,-1.355085
null,0.395681,null,0.39548
0.620102,-1.468948,1.766922,-1.469441
0.354738,-0.834784,0.233956,-0.835179
-1.146126,0.201755,0.710931,0.201758


In [37]:
# New in v0.3.5
# This way, we don't have a reference column, so we cannot respect nulls, but is more convenient to use.
df.with_columns(
    pds.random().alias("[0, 1)"),
    pds.random_normal(pl.col("a").mean(), pl.col("a").std()).alias("Normal"),
    pds.random_int(0, 10).alias("Int from [0, 10)"),
).head()

a,"[0, 1)",Normal,"Int from [0, 10)"
f64,f64,f64,i32
null,0.537011,-0.593783,2
null,0.556977,-1.17522,7
0.620102,0.74351,0.977437,9
0.354738,0.722634,-1.324135,7
-1.146126,0.192882,-0.144544,8


In [38]:
# Genenrate 2 random sample, both normally distributed
# Run Welch's t test on them, p value should be big since they have equal mean
# Run a normality test. Again, p value should be big since they are normally distributed 

df.with_columns(
    pds.random_normal(0.5, 1.0).alias("test1"),
    pds.random_normal(0.5, 2.0).alias("test2"),
).select(
    pds.ttest_ind("test1", "test2", equal_var=False).alias("t-test"),
    pds.normal_test("test1").alias("normality_test")
).select(
    pl.col("t-test").struct.field("statistic").alias("t-tests: statistics")
    , pl.col("t-test").struct.field("pvalue").alias("t-tests: pvalue")
    , pl.col("normality_test").struct.field("statistic").alias("normality_test: statistics")
    , pl.col("normality_test").struct.field("pvalue").alias("normality_test: pvalue")
)

t-tests: statistics,t-tests: pvalue,normality_test: statistics,normality_test: pvalue
f64,f64,f64,f64
0.806691,0.419973,1.536018,0.463936


In [39]:
size = 5_000
df = pl.DataFrame({
    "market_id": range(size),
}).with_columns(
    pl.col("market_id").mod(3),
    var1 = pds.random(),
    var2 = pds.random(),
    category_1 = pds.random_int(0, 5),
    category_2 = pds.random_int(0, 10),
)

df.head(5)

market_id,var1,var2,category_1,category_2
i64,f64,f64,i32,i32
0,0.760853,0.166417,2,8
1,0.149607,0.886886,2,1
2,0.378087,0.709648,2,1
0,0.012671,0.785445,3,3
1,0.451061,0.8895,3,1


In [40]:
# In dataframe statistical tests!
df.select(
    pds.ttest_ind("var1", "var2", equal_var=True).alias("t-test"),
    pds.chi2("category_1", "category_2").alias("chi2-test"),
    pds.f_test("var1", group = "category_1").alias("f-test")
)

t-test,chi2-test,f-test
struct[2],struct[2],struct[2]
"{-0.236185,0.813294}","{44.257796,0.162397}","{0.284087,0.888447}"


In [41]:
# Can also be done in group by context
print(
    df.group_by("market_id").agg(
        pds.ttest_ind("var1", "var2", equal_var=False).alias("t-test"),
        pds.chi2("category_1", "category_2").alias("chi2-test"),
        pds.f_test("var1", group = "category_1").alias("f-test")
    )
)

shape: (3, 4)
┌───────────┬──────────────────────┬──────────────────────┬─────────────────────┐
│ market_id ┆ t-test               ┆ chi2-test            ┆ f-test              │
│ ---       ┆ ---                  ┆ ---                  ┆ ---                 │
│ i64       ┆ struct[2]            ┆ struct[2]            ┆ struct[2]           │
╞═══════════╪══════════════════════╪══════════════════════╪═════════════════════╡
│ 0         ┆ {-0.035991,0.971292} ┆ {49.261984,0.069371} ┆ {1.242167,0.290986} │
│ 1         ┆ {-1.297004,0.19472}  ┆ {52.860716,0.034598} ┆ {0.558786,0.69263}  │
│ 2         ┆ {0.910669,0.362536}  ┆ {37.555952,0.397746} ┆ {1.407349,0.229106} │
└───────────┴──────────────────────┴──────────────────────┴─────────────────────┘


In [42]:
# Benford's law
df.select(
    first_digit_cnt = pds.query_first_digit_cnt(pl.col("var1")).explode()
).with_columns(
    # This doesn't follow benford's law because it is random data
    first_digit_distribution = pl.col("first_digit_cnt") / pl.col("first_digit_cnt").sum()
)

first_digit_cnt,first_digit_distribution
u32,f64
569,0.1138
585,0.117
555,0.111
533,0.1066
545,0.109
578,0.1156
550,0.11
530,0.106
555,0.111


# Nearest Neighbors Related Tasks

These queries can be very slow when data/dimension gets huge, even when processed in parallel.

In [43]:
import polars_ds as pds
size = 2000
df = pl.DataFrame({
    "id": range(size), 
}).with_columns(
    pds.random().alias("var1"),
    pds.random().alias("var2"),
    pds.random().alias("var3"),
    pds.random().alias("r"),
    (pds.random() * 10).alias("rh"),
    pl.col("id").cast(pl.UInt32)
)

In [44]:
# Get neighbor count. The point itself is always considered a neighbor to itself.
df.with_columns(
    pds.query_nb_cnt(
        pl.col("var1"), "var2", "var3", # Columns used as the coordinates in n-d space, str | pl.Expr 
        r = 0.1, # radius 
        dist = "inf", # L Infinity distance 
        parallel = True 
    ).alias("nb_l_inf_cnt")
).head() 

id,var1,var2,var3,r,rh,nb_l_inf_cnt
u32,f64,f64,f64,f64,f64,u32
0,0.720898,0.991827,0.347226,0.467622,2.156275,10
1,0.104165,0.519944,0.157892,0.895848,5.869907,16
2,0.823754,0.340088,0.019865,0.395699,2.7325,10
3,0.875717,0.577812,0.859156,0.361623,0.193279,18
4,0.766846,0.074721,0.249287,0.666755,9.584758,20


In [45]:
df.with_columns(
    pds.query_nb_cnt(
        "var1", "var2", "var3", # Columns used as the coordinates in n-d space, str | pl.Expr 
        r = pl.col("r"), # radius be an expression too
        dist = "l1", # L 1 distance 
        parallel = True 
    ).alias("nb_l1_r_cnt")
).head()

id,var1,var2,var3,r,rh,nb_l1_r_cnt
u32,f64,f64,f64,f64,f64,u32
0,0.720898,0.991827,0.347226,0.467622,2.156275,137
1,0.104165,0.519944,0.157892,0.895848,5.869907,760
2,0.823754,0.340088,0.019865,0.395699,2.7325,88
3,0.875717,0.577812,0.859156,0.361623,0.193279,110
4,0.766846,0.074721,0.249287,0.666755,9.584758,363


In [46]:
# Get ids of the k nearest neighbors. 
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        index = "id",  # pl.col("id"), str | pl.Expr
        k = 3, 
        dist = "l2", # squared l2
        parallel = True
    ).alias("best friends")
).head() 

id,var1,var2,var3,r,rh,best friends
u32,f64,f64,f64,f64,f64,list[u32]
0,0.720898,0.991827,0.347226,0.467622,2.156275,"[0, 466, … 1831]"
1,0.104165,0.519944,0.157892,0.895848,5.869907,"[1, 1982, … 1117]"
2,0.823754,0.340088,0.019865,0.395699,2.7325,"[2, 665, … 612]"
3,0.875717,0.577812,0.859156,0.361623,0.193279,"[3, 377, … 1429]"
4,0.766846,0.074721,0.249287,0.666755,9.584758,"[4, 1358, … 19]"


In [47]:
# Get all neighbors within radius r, call them best friends
print(

df.select(
    pl.col("id"),
    pds.query_radius_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in 3d space
        index = pl.col("id"),
        r = 0.1, 
        dist = "l2", # actually this is squared l2
        parallel = True
    ).alias("best friends"),
).with_columns( # -1 to remove the point itself
    (pl.col("best friends").list.len() - 1).alias("best friends count")
).head()

)

shape: (5, 3)
┌─────┬───────────────────┬────────────────────┐
│ id  ┆ best friends      ┆ best friends count │
│ --- ┆ ---               ┆ ---                │
│ u32 ┆ list[u32]         ┆ u32                │
╞═════╪═══════════════════╪════════════════════╡
│ 0   ┆ [0, 466, … 229]   ┆ 5                  │
│ 1   ┆ [1, 1982, … 579]  ┆ 6                  │
│ 2   ┆ [2, 665, … 1288]  ┆ 7                  │
│ 3   ┆ [3, 377, … 86]    ┆ 9                  │
│ 4   ┆ [4, 1358, … 1104] ┆ 15                 │
└─────┴───────────────────┴────────────────────┘


In [48]:
# Get ids of the k nearest neighbors and distances
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        index = pl.col("id"),
        k = 3, 
        dist = "l2", # actually this is squared l2
        parallel = True,
        return_dist = True
    ).alias("best_friends_w_dist")
).unnest("best_friends_w_dist").head()

id,var1,var2,var3,r,rh,idx,dist
u32,f64,f64,f64,f64,f64,list[u32],list[f64]
0,0.720898,0.991827,0.347226,0.467622,2.156275,"[0, 466, … 1831]","[0.0, 0.059742, … 0.087801]"
1,0.104165,0.519944,0.157892,0.895848,5.869907,"[1, 1982, … 1117]","[0.0, 0.047833, … 0.070508]"
2,0.823754,0.340088,0.019865,0.395699,2.7325,"[2, 665, … 612]","[0.0, 0.042865, … 0.061837]"
3,0.875717,0.577812,0.859156,0.361623,0.193279,"[3, 377, … 1429]","[0.0, 0.070834, … 0.079681]"
4,0.766846,0.074721,0.249287,0.666755,9.584758,"[4, 1358, … 19]","[0.0, 0.018153, … 0.04832]"


In [49]:
# Filter to only points near the given point
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        pt = [0.5, 0.5, 0.5],
        r = 0.2,
        dist = "l2" # actually this is squared l2, so this is asking for squared l2 <= 0.2
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
8,0.441137,0.19478,0.183594,0.398282,9.18641
12,0.734951,0.649102,0.500223,0.883317,9.904633
15,0.108074,0.469015,0.479899,0.918489,9.804659
16,0.485801,0.530277,0.590574,0.706186,6.602236
20,0.360227,0.713703,0.231448,0.138263,3.325791


In [50]:
# Haversine distance is available when dimension is 2
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), # Columns used as the coordinates in n-d space
        pt = [0.5, 0.5],
        r = 10, # in km
        dist = "h" 
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
16,0.485801,0.530277,0.590574,0.706186,6.602236
172,0.486064,0.434635,0.266546,0.299513,3.402964
178,0.501763,0.522313,0.934925,0.709687,1.89558
184,0.543118,0.529606,0.545816,0.0628,3.587051
189,0.494074,0.571409,0.414045,0.330102,6.852999


In [51]:
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), 
        pt = [0.5, 0.5],
        # radius can also be an existing column in the dataframe.
        r = pl.col("rh"), 
        dist = "h" 
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
16,0.485801,0.530277,0.590574,0.706186,6.602236
303,0.502767,0.51603,0.978096,0.738166,9.051765
418,0.508616,0.476217,0.221188,0.421526,7.903868
467,0.489657,0.483175,0.523553,0.093809,7.757587
481,0.516688,0.550573,0.866126,0.848545,9.566443


In [52]:
friends = df.select(
    pl.col("id").cast(pl.UInt64),
    pds.query_radius_ptwise(
        # Columns used as the coordinates in n-d space
        pl.col("var1"), pl.col("var2"), 
        index=pl.col("id"),
        r = 0.02, 
        dist = "l2",
    ).alias("friends")
).with_columns(
    pl.col("friends").list.len().alias("count")
)
friends.head()

id,friends,count
u64,list[u32],u32
0,"[0, 1421, 1777]",3
1,"[1, 17, … 842]",4
2,"[2, 1857, … 665]",4
3,"[3, 1868, 1468]",3
4,"[4, 982, … 774]",8


# Using PDS Expressions On pl.Series, NumPy arrays, or pd.Series, etc.

The output by default is always a Polars Series. The user gets to choose whether to turn it into NumPy, Pandas, or other data structures.

In [53]:
import pandas as pd
import numpy as np
from polars_ds.compat import compat as pds2

df = pds.frame(size=100_000).select(
    pds.random(0.0, 1.0).round().alias("actual"),
    pds.random(0.0, 1.0).alias("predicted"),
    pds.random_int(0, 3).alias("0-2"),
    pds.random_int(0, 10).alias("0-9"),
    pds.random_str(min_size=1, max_size=2).alias("s1"),
    pds.random_str(min_size=1, max_size=2).alias("s2"),
)
df.head()

/tmp/ipykernel_23937/3354819425.py:3: UserWarning: The compatibility layer is considered experimental.
  from polars_ds.compat import compat as pds2


actual,predicted,0-2,0-9,s1,s2
f64,f64,i32,i32,str,str
0.0,0.192482,2,2,"""Mk""","""DR"""
1.0,0.010681,2,8,"""en""","""9"""
1.0,0.096006,0,6,"""R""","""Q"""
1.0,0.238899,0,2,"""m""","""t"""
0.0,0.451355,1,8,"""x8""","""a"""


In [54]:
df_pd = df.to_pandas()

In [55]:
# Pandas Series
pds2.jaccard_col(df_pd["0-2"], df_pd["0-9"])

jaccard_col
f64
0.3


In [56]:
# Polars Series
print(pds2.query_binary_metrics(df["actual"], df["predicted"]).struct.unnest())
# Pyarrow Series
print(pds2.query_binary_metrics(df["actual"].to_arrow(), df["predicted"].to_arrow()).struct.unnest())
# NumPy
print(pds2.query_binary_metrics(df["actual"].to_numpy(), df["predicted"].to_numpy()).struct.unnest())

shape: (1, 5)
┌───────────┬──────────┬──────────┬───────────────────┬──────────┐
│ precision ┆ recall   ┆ f        ┆ average_precision ┆ roc_auc  │
│ ---       ┆ ---      ┆ ---      ┆ ---               ┆ ---      │
│ f64       ┆ f64      ┆ f64      ┆ f64               ┆ f64      │
╞═══════════╪══════════╪══════════╪═══════════════════╪══════════╡
│ 0.497157  ┆ 0.496392 ┆ 0.496774 ┆ 0.498906          ┆ 0.496817 │
└───────────┴──────────┴──────────┴───────────────────┴──────────┘
shape: (1, 5)
┌───────────┬──────────┬──────────┬───────────────────┬──────────┐
│ precision ┆ recall   ┆ f        ┆ average_precision ┆ roc_auc  │
│ ---       ┆ ---      ┆ ---      ┆ ---               ┆ ---      │
│ f64       ┆ f64      ┆ f64      ┆ f64               ┆ f64      │
╞═══════════╪══════════╪══════════╪═══════════════════╪══════════╡
│ 0.497157  ┆ 0.496392 ┆ 0.496774 ┆ 0.498906          ┆ 0.496817 │
└───────────┴──────────┴──────────┴───────────────────┴──────────┘
shape: (1, 5)
┌───────────┬───────

In [57]:
# NumPy Arrays
pds2.psi(
    np.random.random(size = 1000), 
    np.random.random(size = 1000), 
    n_bins = 5, 
    return_report = True
).struct.unnest()

<=,baseline_pct,actual_pct,psi_bin
f64,f64,f64,f64
0.198333,0.2,0.216,0.001231
0.412338,0.2,0.218,0.001551
0.597273,0.2,0.174,0.003621
0.784626,0.2,0.184,0.001334
inf,0.2,0.208,0.000314


In [58]:
pds2.query_cid_ce(
    np.random.random(size = 1000),
)

cid_ce
f64
13.47064


In [59]:
pds2.query_c3_stats(
    pl.Series(values=np.random.random(size = 1000)), 
    lag = 3
)

c3_stats
f64
0.141005


In [60]:
pds2.str_leven(df["s1"], df["s2"]).head() 

str_leven
u32
2
2
1
1
2
1
1
2
2
